In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


import required libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.optimizers import Adam,RMSprop
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

import warnings
warnings.filterwarnings('ignore')

import dataset

In [3]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

setting x as our independent variables and y as our dependent variable

In [4]:
x = train_df.drop(['label'],axis=1)
y = train_df['label']

deep learning performs well on big datasets so importing same dataset of digit from mnist to merge with our current dataset

In [5]:
from keras.datasets import mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In ANN 1D array is used. so reshaping (1,28,28) to (1,784) 

In [6]:
x_train1=np.concatenate([x_train,x_test],axis=0)
y_train1=np.concatenate([y_train,y_test],axis=0)
x_train2 = x_train1.reshape(-1,28*28)
y_train2= y_train1

In [7]:
x_train3=pd.DataFrame(x_train2,columns=[i for i in x.columns])

concatenating two datasets

In [8]:
x1 = pd.concat([x,x_train3],axis=0,ignore_index=True)
y1 = pd.concat([y,pd.DataFrame(y_train2)],axis=0,ignore_index=True)

Scaling the values of independent variables(x)

In [9]:
x1 = x1/255

model implementation

In [10]:
model = Sequential()

2023-02-12 08:09:41.454430: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
# input layer
# using elu because relu gives a dying relu problem
# using regularizer to reduce complexity of the layer
model.add(Dense(128,activation='elu',input_dim = 784,kernel_regularizer=tensorflow.keras.regularizers.L2(0.001),kernel_initializer='he_normal' ))
model.add(BatchNormalization())
model.add(Dropout(0.1))

# hidden layer
model.add(Dense(64,activation='elu',kernel_regularizer=tensorflow.keras.regularizers.L2(0.001),kernel_initializer='he_normal'))
# using BatchNormalization() to normalize the output of this layer
model.add(BatchNormalization())
model.add(Dropout(0.1))

# hidden layer
model.add(Dense(32,activation='elu',kernel_regularizer=tensorflow.keras.regularizers.L2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
# using Dropout to randomly drop 10% neurons in this layer
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(16,activation='elu',kernel_regularizer=tensorflow.keras.regularizers.L2(0.001),kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# output layer
model.add(Dense(10,activation='softmax'))

In [12]:
model.compile(optimizer='RMSprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'],)

In [13]:
# using callback to stop epoch when overfitting tends to start
# using batch_size to divide set of rows into batches, where each batch contains 256 rows
callback = EarlyStopping(monitor='val_accuracy',min_delta= 0.001,patience=15,verbose=0,mode='auto',baseline=None,restore_best_weights=True)
model.fit(x1,y1,epochs=1000,batch_size=256,callbacks=callback,validation_split=0.2)

2023-02-12 08:09:42.439749: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/1000
350/350 [==============================] - 5s 10ms/step - loss: 0.8751 - accuracy: 0.8482 - val_loss: 0.4586 - val_accuracy: 0.9388
Epoch 2/1000
350/350 [==============================] - 3s 10ms/step - loss: 0.4840 - accuracy: 0.9267 - val_loss: 0.3229 - val_accuracy: 0.9609
Epoch 3/1000
350/350 [==============================] - 3s 10ms/step - loss: 0.3781 - accuracy: 0.9421 - val_loss: 0.2635 - val_accuracy: 0.9687
Epoch 4/1000
350/350 [==============================] - 3s 9ms/step - loss: 0.3292 - accuracy: 0.9489 - val_loss: 0.2312 - val_accuracy: 0.9717
Epoch 5/1000
350/350 [==============================] - 3s 10ms/step - loss: 0.2958 - accuracy: 0.9525 - val_loss: 0.2090 - val_accuracy: 0.9742
Epoch 6/1000
350/350 [==============================] - 4s 10ms/step - loss: 0.2702 - accuracy: 0.9560 - val_loss: 0.2058 - val_accuracy: 0.9719
Epoch 7/1000
350/350 [==============================] - 4s 11ms/step - loss: 0.2533 - accuracy: 0.9581 - val_loss: 0.1827 - val_acc

In [14]:
# predict results
test_df = test_df/255
results = model.predict(test_df)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [15]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("sample_submission.csv",index=False)